In [66]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [75]:
df = pd.read_csv('Test_2.csv',header=None)
data = df.drop(df.index[[0,1]])
col_names = list(df.iloc[1,1:])
col_names.insert(0,'Stock Index')
print(col_names)
data.columns = col_names
data.head()


['Stock Index', '10/Aug', '11/Aug', '12/Aug', '13/Aug', '14/Aug', '15/Aug']


,Stock Index,10/Aug,11/Aug,12/Aug,13/Aug,14/Aug,15/Aug
2,AC3235,0.8,0.66,0.7724,0.983224,0.90121088,0.8983682944
3,AC3236,0.86,0.732,0.85448,1.0866448,1.017042176,1.02925765888
4,AC3237,0.93,0.816,0.95024,1.2073024,NaN,1.18196191744
5,AC3238,1.09,1.008,1.16912,1.4830912,1.461062144,1.53100022272
6,AC3239,NaN,1.104,1.27856,1.6209856,1.615503872,1.70551937536


In [76]:
data = data.drop(['Stock Index'],axis=1)
data.head()

,10/Aug,11/Aug,12/Aug,13/Aug,14/Aug,15/Aug
2,0.8,0.66,0.7724,0.983224,0.90121088,0.8983682944
3,0.86,0.732,0.85448,1.0866448,1.017042176,1.02925765888
4,0.93,0.816,0.95024,1.2073024,NaN,1.18196191744
5,1.09,1.008,1.16912,1.4830912,1.461062144,1.53100022272
6,NaN,1.104,1.27856,1.6209856,1.615503872,1.70551937536


In [77]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

imp = IterativeImputer(max_iter=10, random_state=0)
data = imp.fit_transform(data)
print(data)
print(np.amax(data))

[[0.8        0.66       0.7724     0.983224   0.90121088 0.89836829]
 [0.86       0.732      0.85448    1.0866448  1.01704218 1.02925766]
 [0.93       0.816      0.95024    1.2073024  1.15217888 1.18196192]
 ...
 [0.67       0.504      0.59456    0.7591456  0.65024307 0.61477467]
 [0.79       0.648      0.75872    0.9659872  0.88190566 0.8765534 ]
 [0.74       0.588      0.69032    0.8798032  0.78537958 0.76747893]]
1.96729810432


In [78]:
print(data.shape)

(3331, 6)


In [79]:
X = data[:,0:-1]
y = data[:,[-1]]

In [80]:
#splitting dataset into train and test split

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X,y,train_size=0.7)

In [81]:
print(x_train.shape)
print(x_test.shape)

(2331, 5)
(1000, 5)


In [85]:
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], 1)
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], 1)

In [86]:
#LSTM model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

model = Sequential()
model.add(LSTM(128,return_sequences=True,input_shape=(5,1)))
model.add(LSTM(128,return_sequences=True))
model.add(LSTM(128))
model.add(Dense(1))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 5, 128)            66560     
_________________________________________________________________
lstm_4 (LSTM)                (None, 5, 128)            131584    
_________________________________________________________________
lstm_5 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 329,857
Trainable params: 329,857
Non-trainable params: 0
_________________________________________________________________


In [88]:
model.compile(loss = 'mean_squared_error', optimizer = 'adam', metrics=['mse','mae'])

model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=100,batch_size=64,verbose=1)

Train on 2331 samples, validate on 1000 samples
Epoch 1/100
2331/2331 [==============================] - 7s 3ms/sample - loss: 0.3160 - mse: 0.3160 - mae: 0.4128 - val_loss: 0.0494 - val_mse: 0.0494 - val_mae: 0.1853
Epoch 2/100
2331/2331 [==============================] - 1s 422us/sample - loss: 0.0241 - mse: 0.0241 - mae: 0.1260 - val_loss: 0.0060 - val_mse: 0.0060 - val_mae: 0.0642
Epoch 3/100
2331/2331 [==============================] - 1s 283us/sample - loss: 0.0021 - mse: 0.0021 - mae: 0.0334 - val_loss: 7.7874e-04 - val_mse: 7.7874e-04 - val_mae: 0.0234
Epoch 4/100
2331/2331 [==============================] - 1s 303us/sample - loss: 8.0443e-04 - mse: 8.0443e-04 - mae: 0.0243 - val_loss: 7.7654e-04 - val_mse: 7.7654e-04 - val_mae: 0.0215
Epoch 5/100
2331/2331 [==============================] - 1s 310us/sample - loss: 6.9693e-04 - mse: 6.9693e-04 - mae: 0.0222 - val_loss: 6.1906e-04 - val_mse: 6.1906e-04 - val_mae: 0.0207
Epoch 6/100
2331/2331 [==============================] - 1s

2331/2331 [==============================] - 1s 365us/sample - loss: 5.8359e-06 - mse: 5.8359e-06 - mae: 0.0021 - val_loss: 5.6587e-06 - val_mse: 5.6587e-06 - val_mae: 0.0020
Epoch 45/100
2331/2331 [==============================] - 1s 346us/sample - loss: 6.9123e-06 - mse: 6.9123e-06 - mae: 0.0022 - val_loss: 6.7579e-06 - val_mse: 6.7579e-06 - val_mae: 0.0021
Epoch 46/100
2331/2331 [==============================] - 1s 336us/sample - loss: 7.5949e-06 - mse: 7.5949e-06 - mae: 0.0022 - val_loss: 5.9681e-06 - val_mse: 5.9681e-06 - val_mae: 0.0020
Epoch 47/100
2331/2331 [==============================] - 1s 363us/sample - loss: 7.1542e-06 - mse: 7.1542e-06 - mae: 0.0022 - val_loss: 5.9689e-06 - val_mse: 5.9689e-06 - val_mae: 0.0020
Epoch 48/100
2331/2331 [==============================] - 1s 357us/sample - loss: 6.6913e-06 - mse: 6.6913e-06 - mae: 0.0021 - val_loss: 5.3219e-06 - val_mse: 5.3219e-06 - val_mae: 0.0020
Epoch 49/100
2331/2331 [==============================] - 1s 304us/sample

2331/2331 [==============================] - 1s 327us/sample - loss: 7.4564e-06 - mse: 7.4564e-06 - mae: 0.0022 - val_loss: 9.7427e-06 - val_mse: 9.7427e-06 - val_mae: 0.0030
Epoch 88/100
2331/2331 [==============================] - 1s 334us/sample - loss: 2.9952e-05 - mse: 2.9952e-05 - mae: 0.0045 - val_loss: 1.7352e-05 - val_mse: 1.7352e-05 - val_mae: 0.0040
Epoch 89/100
2331/2331 [==============================] - 1s 337us/sample - loss: 1.4009e-04 - mse: 1.4009e-04 - mae: 0.0099 - val_loss: 2.0780e-05 - val_mse: 2.0780e-05 - val_mae: 0.0044
Epoch 90/100
2331/2331 [==============================] - 1s 334us/sample - loss: 1.8552e-05 - mse: 1.8552e-05 - mae: 0.0034 - val_loss: 6.8689e-06 - val_mse: 6.8689e-06 - val_mae: 0.0024
Epoch 91/100
2331/2331 [==============================] - 1s 322us/sample - loss: 1.8608e-05 - mse: 1.8608e-05 - mae: 0.0032 - val_loss: 1.1707e-06 - val_mse: 1.1707e-06 - val_mae: 8.9247e-04
Epoch 92/100
2331/2331 [==============================] - 1s 324us/sa

In [92]:
from sklearn.metrics import mean_squared_error
x_train_pred = model.predict(x_train)
train_rmse = mean_squared_error(y_train,x_train_pred,squared=False)
print(train_rmse)


x_test_pred = model.predict(x_test)
test_rmse = mean_squared_error(y_test,x_test_pred,squared=False)
print(test_rmse)

0.0015886856308180177
0.0015302156019993182


In [100]:
#Put-Call Ratio prediction for 16th Aug

x_input = data[:,1:]
print(x_input.shape)
x_input = x_input.reshape(x_input.shape[0],x_input.shape[1],1)
print(x_input.shape)
y_input = model.predict(x_input)
print('Put-Call Ratio prediction for 16th Aug','\n',y_input)

(3331, 5)
(3331, 5, 1)
Put-Call Ratio prediction for 16th Aug 
 [[0.9115969 ]
 [1.0663332 ]
 [1.2474985 ]
 ...
 [0.57771623]
 [0.8858633 ]
 [0.7573909 ]]


In [101]:
np.savetxt("Put-Call Ratio.csv", y_input, delimiter=",")